In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [3]:
pyspark.__version__

'3.3.2'

In [4]:
spark = SparkSession.builder \
        .master("local[*]") \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/07 17:25:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
!pwd

/home/dheeru_user/notebooks


In [6]:
df = spark.read\
    .option("header","true")\
    .parquet("/home/dheeru_user/spark/data/homework/yellow_tripdata_2024-10/yellow_tripdata_2024-10.parquet")

In [7]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [8]:
df = df.repartition(4)

In [9]:
!pwd

/home/dheeru_user/notebooks


In [11]:
df.write.parquet(path='/home/dheeru_user/spark/data/homework/yellow_tripdata_2024-10_partitioned', mode="overwrite")

In [12]:
df_yellow = spark.read.parquet('/home/dheeru_user/spark/data/homework/yellow_tripdata_2024-10_partitioned/*')

In [13]:
from pyspark.sql import functions as F

In [14]:
df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-10 10:34:57|  2024-10-10 10:53:48|              1|          3.4|         1|                 N|         140|          74|           1|       20.5|  0.0|    0.5|      0.2

In [15]:
df_yellow.registerTempTable('yellow_trip')

/home/dheeru_user/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [16]:
spark.sql("""
SELECT
    COUNT(*)
FROM
    yellow_trip
WHERE
    DATE(tpep_pickup_datetime) = '2024-10-15'
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [17]:
spark.sql("""
SELECT
    MAX(TIMESTAMPDIFF(HOUR, tpep_pickup_datetime, tpep_dropoff_datetime)) as longest_trip
FROM
    yellow_trip
""").show()

+------------+
|longest_trip|
+------------+
|         162|
+------------+



In [19]:
df_zone = spark.read\
            .option("header", "true")\
            .csv("/home/dheeru_user/spark/data/homework/taxi_zone_lookup.csv")

In [20]:
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [21]:
df_zone.registerTempTable('zone')

In [28]:
spark.sql("""
SELECT
    Zone,
    COUNT(1) as frequency
FROM
    yellow_trip y
    JOIN zone z
    ON y.PULocationID = z.LocationID
GROUP BY
    Zone
ORDER BY
    frequency ASC
LIMIT
    1
""").show()

+--------------------+---------+
|                Zone|frequency|
+--------------------+---------+
|Governor's Island...|        1|
+--------------------+---------+

